# Prerequisite
* 環境要先安裝fastText: https://github.com/facebookresearch/fastText
* 環境要先安裝Sent2Vec: https://github.com/epfml/sent2vec 

In [ ]:
import os,sys
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

決定要採用那些API calls (要先自己統計dataset中出現了哪一些API function name)

In [ ]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #dataset中少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?小寫?
len(api_li)

## 準備Sent2Vec演算法所需要的訓練資料集

In [ ]:
corpus = []
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" 

root_dir = './data/tree-rep-profiles_o2o/normal/' #訓練資料集
# rasmma_dir  = next(os.walk(root_dir))[1]
# for dir_ in rasmma_dir:
fam_dir = next(os.walk(root_dir ))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir  + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for profile in hl_list:
            with open(profile,encoding='ISO 8859-1') as f:
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
            lines = re.sub('=\\n','',lines)
            lines = re.sub('y\\n','',lines)
            lines = lines.splitlines()
            for line in lines:
                temp = re.sub(dil," ",line.lower())#.lower()) #小寫?
                temp = temp.split(" ")
                temp = list(filter(None, temp))
                temp = ' '.join(temp)
                if temp.startswith('winh'.lower()):
                    print(profile,temp)
                if temp.split(' ')[0] not in api_li:
                    print('=o2o_ERR:=',profile,'=>',temp) #API function name不正確，通常是編碼或是亂碼問題
                    continue
                corpus.append(temp)

with open('data/sent2vec_corpus/o2o_Sent2Vec_lower_woParam_0630.txt','w') as f: #輸出corpus給Sent2Vec用，要先自行創好資料夾
    f.write('\n'.join(corpus))
corpus[-10:] #範例

有了corpus以後就可以依照https://github.com/epfml/sent2vec#train-a-new-sent2vec-model 進行訓練
* 超參數說明皆未於Sent2Vec github
* 超參數選擇可以依照以下的evaluation方式來挑選調整
    *  TY自己tune出來的超參數: -minCount 0 -dim 768 -epoch 15 -lr 0.3 -wordNgrams 16 -loss ns -neg 10 -thread 30 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2

# Embedding evaluation

In [ ]:
with open('data/sent2vec_corpus/o2o_Sent2Vec_lower_woParam_0630.txt','r') as f: #讀入sent2vec訓練用的corpus
    corpus = f.read().splitlines() 
corpus = sorted(corpus)
corpus[-100:]

In [ ]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_woParam_0616.bin') #讀入最終訓練完成的Sent2Vec model
emb_all = model.embed_sentences(corpus)

In [ ]:
# 每一種API finctoin name出現了幾次?
api = ""
count = 0
count_all = {}
for row in corpus:
    try:
        count_all[row.split(' ')[0]] += 1
    except KeyError:
        count_all[row.split(' ')[0]] = 1
print(len(count_all))
sorted(count_all.items())

In [ ]:
# 少了那些該出現的API function name?
# 多了哪些不該出現的API function name?
kk = list(count_all.keys())
for k in api_li:
    if k not in kk:
        print('少了:',k)
for k in kk:
    if k not in api_li:
        print('多了',k)

以cosine similarity方式評估，隨機挑選100筆，看自己的API function name可否還原到自己那一類

In [ ]:
c= 0
df = pd.DataFrame(0, index=api_li, columns=api_li)
index_ = df.index.tolist()
avg = 100
for _ in range(avg):
    c=0
    for k,v in count_all.items():
        pick = random.randrange(c,c+v) #cosine similarity的左邊
        pick = emb_all[pick,:] #sorted corpus的embedding_all
        c1 = 0
        try:
            row = index_.index(k)
        except ValueError:
            continue
        for k1,v1 in count_all.items(): # cosine similarity的右邊
            pick1 = random.randrange(c1,c1+v1)
            pick1 = emb_all[pick1,:]
            score = cosine_similarity([pick],[pick1]) #cosine_similarity
    #         print(score)
            c1 = c1+v1
            try:
                col = index_.index(k1)
                df.iloc[row,col] += float(score)
            except ValueError:
                continue
        c = c+v
df = df/avg #similarity
# df = (df-np.min(df.values))/(np.max(df.values)-np.min(df.values)) #distance normalize
df

In [ ]:
lib = ['LoadLibrary']
proc = ['CreateProcess', 'OpenProcess', 'ExitProcess', 'TerminateProcess', 'WinExec',
        'CreateRemoteThread', 'CreateThread']
file = ['CopyFile', 'CreateFile', 'DeleteFile']
reg = ['RegSetValue', 'RegCreateKey', 'RegDeleteKey', 'RegDeleteValue',
       'RegQueryValue', 'RegEnumValue']
net = ['WinHttpConnect', 'WinHttpOpen', 'WinHttpOpenRequest', 'WinHttpReadData', 'WinHttpSendRequest', #'WinHttpWriteData',
        'InternetOpen', 'InternetConnect', 'HttpSendRequest', 'GetUrlCacheEntryInfo']

#lower?小寫?要跟前面一致
lib = [x.lower() for x in lib]
proc = [x.lower() for x in proc]
file = [x.lower() for x in file]
reg =[x.lower() for x in reg]
net = [x.lower() for x in net]

# df.loc['']
x = [lib,proc,file,reg,net]
x = sum(x,[])
index_ , len(index_)

In [ ]:
up=0
down = len(index_)
down_cat = len(x)
up_cat = 0
up_cat2 = 0
lib_lib=lib_proc=lib_file=lib_reg=lib_net = 0
proc_lib=proc_proc=proc_file=proc_reg=proc_net = 0
file_lib=file_proc=file_file=file_reg=file_net = 0
reg_lib=reg_proc=reg_file=reg_reg=reg_net = 0
net_lib=net_proc=net_file=net_reg=net_net = 0
for api in index_:
    if df.loc[api].idxmax() == api:
        up+=1
    else:
        print(api)
    baseline_lib = df.loc[api,lib].mean()
    baseline_proc =  df.loc[api,proc].mean()
    baseline_file = df.loc[api,file].mean()
    baseline_reg = df.loc[api,reg].mean()
    baseline_net = df.loc[api,net].mean()
    max_value = np.max([baseline_lib,baseline_proc,baseline_file,baseline_reg,baseline_net])
    if (api in lib):
        lib_lib += baseline_lib
        lib_proc += baseline_proc
        lib_file += baseline_file
        lib_reg += baseline_reg
        lib_net += baseline_net
        if (max_value==baseline_lib):
            up_cat+=1
    elif api in proc :
        proc_lib += baseline_lib
        proc_proc += baseline_proc
        proc_file += baseline_file
        proc_reg += baseline_reg
        proc_net += baseline_net
        if max_value==baseline_proc:
            up_cat+=1
    elif api in file :
        file_lib += baseline_lib
        file_proc += baseline_proc
        file_file += baseline_file
        file_reg += baseline_reg
        file_net += baseline_net
        if max_value==baseline_file:
            up_cat+=1    
    elif api in reg:
        reg_lib += baseline_lib
        reg_proc += baseline_proc
        reg_file += baseline_file
        reg_reg += baseline_reg
        reg_net += baseline_net
        if max_value==baseline_reg:
            up_cat+=1    
    elif api in net:
        net_lib += baseline_lib
        net_proc += baseline_proc
        net_file += baseline_file
        net_reg += baseline_reg
        net_net += baseline_net
        if max_value==baseline_net:
            up_cat+=1
if np.max([lib_lib,lib_proc,lib_file,lib_reg,lib_net]) == lib_lib:
    up_cat2+=1
if np.max([proc_lib,proc_proc,proc_file,proc_reg,proc_net]) == proc_proc:
    up_cat2+=1
if np.max([file_lib,file_proc,file_file,file_reg,file_net]) == file_file:
    up_cat2+=1
if np.max([reg_lib,reg_proc,reg_file,reg_reg,reg_net]) == reg_reg:
    up_cat2+=1
if np.max([net_lib,net_proc,net_file,net_reg,net_net]) == net_net:
    up_cat2+=1
        
print('自己還原自己-ACC:',(up/down)*100,'%') #self
print('自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的):',(up_cat/down_cat)*100,'%') #micro
print('自己那類屬於自己那一類-ACC:',(up_cat2/5)*100,'%') #macro

In [ ]:
# 綜合性決定性評估指標:
score = []
for i in range(len(df)):
    score.append(df.iloc[i,i])
print('對角線分數(越高越好):',np.mean(score))

TY的Sent2Vec model表現:
* 自己還原自己-ACC: 96.15384615384616 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 73.07692307692307 %
* 自己那類屬於自己那一類-ACC: 100.0 %
* 對角線分數(越高越好): 0.843296217550523

***

# Appendix Spplementary
* 因相關套件版本變動過大而成為Deprecated code. Need lots of modifications!!!!!

## Embedding layer weight initialization
**建議使用他們推薦的model、tokenizer進行訓練: bert-base-multilingual-cased**

1. 需先安裝pytorch與BERT於環境: https://github.com/huggingface/pytorch-transformers#installation、https://huggingface.co/pytorch-transformers/index.html
2. 依照說明創造finetune pre-trained model所需的corpus: https://github.com/huggingface/pytorch-transformers/tree/master/examples/lm_finetuning
3. 依照相同說明Pregenerating training data: https://github.com/huggingface/pytorch-transformers/tree/master/examples/lm_finetuning#pregenerating-training-data
4. 依照相同說明Training on pregenerated data(非常耗時且不可中斷，需要多張GPU): https://github.com/huggingface/pytorch-transformers/tree/master/examples/lm_finetuning#training-on-pregenerated-data

5. 假設讀入兩種經過finetune的BERT model，並利用內建推薦的tokenizer: bert-base-multilingual-cased

In [ ]:
model_path1 = '/DATA/r06725035/model/less_pretrain_bert/bert/pytorch_model.bin' #model1 path
model_state_dict1 = torch.load(model_path1,  map_location='cpu')
model1 = BertModel.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-cased',state_dict=model_state_dict1)
model1 = torch.nn.DataParallel(model1)
model1.eval()

model_path2 = './model/bert_pretrained/bert/pytorch_model.bin' #model2 path
model_state_dict2 = torch.load(model_path2,  map_location='cpu')
model2 = BertModel.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-cased',state_dict=model_state_dict2)
model2 = torch.nn.DataParallel(model2)
model2.eval()
# model = BertModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='bert-base-multilingual-cased', do_lower_case=False)

獲取兩種model最後四層hidden layer所輸出的feature

In [ ]:
def get_feature(hl_path,model1=model1,model2=model2,tokenizer=tokenizer,sent=True):
    '''
    Input: hl_path=> 一個execution profile的路徑位子
    Return: np.array=> 該execution profile在其中一種model所萃取出的768 feature
    '''
    last = [-1,-2,-3,-4] #要拿最後幾層?
    cls_np_all = []
    token_np_all = []
    with open(hl_path,encoding='ISO 8859-1') as f:
        lines = f.read().splitlines()
    for line in lines:
        line = '[CLS] ' + line + ' [SEP]'
        tokenized_text = tokenizer.tokenize(line)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [0]*len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
#         tokens_tensor = tokens_tensor.cuda() #.to(device)
#         segments_tensors = segments_tensors.cuda() #.to(device)
        if sent:
            model1.cuda()
            tokens_tensor = tokens_tensor.cuda()
            segments_tensors = segments_tensors.cuda()
        else:
            model2.cuda()
            tokens_tensor = tokens_tensor.cuda() #.to(device)
            segments_tensors = segments_tensors.cuda() #.to(device)
#         print(len(tokenized_text))
        if sent: #只拿[CLS] 最後四層
            with torch.no_grad():
                encoded_layers, _ = model1(tokens_tensor, segments_tensors)
            cls_np = []
            for layer in last:
                cls_np.append(np.array(encoded_layers[layer].cpu())[0][0])
            last_np = np.sum(cls_np,axis=0)
            cls_np_all.append(last_np)
        else:
            with torch.no_grad():
                encoded_layers, _ = model2(tokens_tensor, segments_tensors)
            length = len(encoded_layers[-1][0])
            all_np = []
            for layer in last:
                for token in range(1,length-1):
                    array = np.array(encoded_layers[layer].cpu())[0][token]
                    all_np.append(array)
            last_np = np.average(all_np,axis=0)
            token_np_all.append(last_np) #只有一個lline!!
    try:
        del model1
        del model2
    except:
        torch.cuda.empty_cache()
    if sent:
        return np.array(cls_np_all) #[CLS]
    else:
        return np.array(token_np_all) # tokens
            
#     break

取得每一個profile兩種model的feature

In [ ]:
#sent=False是二種model，存到XXX_tokenfeature.pkl
#sent=True是第一種model，存到_clsfeature.pkl
#所以要跑兩次
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for hl in hl_list:
            clean_profile(hl)
#         in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile_clean"), hl_list))
        for hl in hl_list:
            tokens_feature = get_feature(hl,model1,model2,sent=False) #回傳該hkl的兩種feature
#             cls_file = hl.replace('.profile_clean','_clsfeature.pkl') #一種model的輸出
            token_file = hl.replace('.profile_clean','_tokenfeature.pkl') #另一種model的輸出
#             pickle.dump(file=open(cls_file,'wb'),obj=cls_feature)
            pickle.dump(file=open(token_file,'wb'),obj=tokens_feature)
            

抓出每一種API function name之所有API invocation call vectors進行平均
* 要先有已經定義好的api_li

In [ ]:
whole_dict ={}
whole_dict_count ={}
pretrain_dict ={}
whole_dict = whole_dict.fromkeys(api_li,np.array([0.0]*768))
pretrain_dict = pretrain_dict.fromkeys(api_li,np.array([0.0]*768))
whole_dict_count = whole_dict.fromkeys(api_li,0)

root_dir = './data/tree-rep-profiles_one2many/normal/' #已經存了各別feature pkl的路徑

# whole_dict = {}
 
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith("_clsfeature.pkl"), hl_list))
        for pkl in hl_list:
            temp = pickle.load(open(pkl,'rb'))
            path2 = pkl.replace('_clsfeature.pkl','.profile_clean')
            with open(path2,encoding='ISO 8859-1') as f:
                lines = f.read().splitlines()
            assert len(temp) == len(lines)
            for num,line in zip(temp,lines):
                api = line.split(' ')[0]
#                 if api == 'GetUrlCacheEntryInfo':
#                     print(api)
                whole_dict[api] = whole_dict[api]+num
                whole_dict_count[api] = whole_dict_count[api] + 1
whole_dict_count , whole_dict['LoadLibrary']

In [ ]:
for api in api_li:
    pretrain_dict[api] = whole_dict[api] / (whole_dict_count[api] + 1e-8)
# pretrain_dict[api]

embedding_matrix = np.zeros((len(pretrain_dict) + 1, 768))
for word, i in encode_dict.items():
    embedding_vector = pretrain_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
pickle.dump(file=open('data/tree-rep-profiles-partial/api_emb_matrix.pkl','wb'),obj=embedding_matrix) 
#儲存每一種API function name的initial weight vector for embedding layer
# embedding_matrix